# Initialisation

In [58]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
import tifffile
%matplotlib tk

# Importer fichier

In [59]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [60]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [61]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




1


# Traitement d'image

In [62]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)
print(img)

[[ 9 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 [ 9  9 10 ... 11 12 12]
 ...
 [ 9  9 10 ...  8  8  8]
 [ 8  9  9 ...  8  8  8]
 [ 8  9  9 ...  8  9  8]]


# Sélection du point à tracker

In [63]:
def crop(img, x, y, crop_size=100):
    x_start = int(x - crop_size // 2)
    x_end = int(x + crop_size // 2)
    y_start = int(y - crop_size // 2)
    y_end = int(y + crop_size // 2)

    return img[y_start:y_end, x_start:x_end]

In [64]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = 100

Please click on the point you want to select.
Selected point: (731.1883116883117, 395.1493506493505)


Debogueur

In [65]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [66]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [67]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [68]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + maxi[0] - crop_sze/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [69]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Faire le crop et fit

In [70]:
def particle_tracker(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze // 2 + max_coords[1]
    nouveau_y = y - crop_sze // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [x, y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [ ]:
position_list=[]
sigma_list = []
crop_frames = []
big_frames = []


for i in range(first_frame,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
    else:
        data = particle_tracker(img, x, y)
        position_list.append(data[2])
        x,y = position_list[i-first_frame]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])



hello
hello
hello


In [72]:
pixel_size = 3.45
sigma_arr=np.array(sigma_list)* pixel_size * 10**(-6)
position_arr=np.array(position_list)* pixel_size * 10**(-6)
print(position_arr)

[[0.00252646 0.00137317]
 [0.00252874 0.00138438]
 [0.00253262 0.00139866]
 [0.00253216 0.00140224]
 [0.00253169 0.0014132 ]
 [0.00253177 0.00142287]
 [0.00253188 0.00142854]
 [0.00253349 0.00144142]
 [0.00253897 0.00145021]
 [0.00254171 0.00145972]
 [0.00254366 0.0014668 ]
 [0.00254747 0.00147536]
 [0.00255298 0.00148211]
 [0.00255635 0.00148991]
 [0.00255591 0.00149089]
 [0.00255818 0.00150047]
 [0.00256077 0.00150249]
 [0.00255898 0.00151758]
 [0.00255746 0.00152112]
 [0.00256217 0.00152135]
 [0.00256545 0.00152889]
 [0.00256941 0.00151107]
 [0.00257131 0.00151609]
 [0.00257455 0.00151763]
 [0.00257239 0.00151572]
 [0.00257467 0.00151352]
 [       nan        nan]
 [0.00257674 0.00152167]
 [       nan        nan]
 [0.00257967 0.0015284 ]
 [0.00257853 0.00151933]
 [0.00258033 0.00152827]
 [0.00258152 0.00153034]
 [0.00258382 0.00153067]
 [0.00258472 0.00152807]
 [0.00257802 0.00152479]
 [0.00257615 0.00151989]
 [0.00255588 0.00152244]
 [0.00255802 0.00152251]
 [0.00255856 0.00152319]


In [73]:
def calculate_msd_with_uncertainty(position, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """


    msd = []
    uncertainties = []
    for d in range(1, len(position_arr)):
        diff_pairs = np.zeros_like(position_arr[d:])
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(position_arr[d:]) | np.isnan(position_arr[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta
        
        diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, :] - position_arr[:-d][valid_mask_pos, :]

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([4.17296287e-10, 8.81814828e-10, 1.38078819e-09, 1.92890213e-09,
        2.51163576e-09, 3.10164949e-09, 3.70094693e-09, 4.43684276e-09,
        5.14999957e-09, 5.99388631e-09, 6.77265793e-09, 7.52898451e-09,
        8.28367992e-09, 9.00329970e-09, 9.77510342e-09, 9.95346947e-09,
        1.04014195e-08, 1.04320617e-08, 1.09759322e-08, 1.14124035e-08,
        1.18401230e-08, 1.23432983e-08, 1.28767533e-08, 1.32507095e-08,
        1.36624089e-08, 1.41612595e-08, 1.49459576e-08, 1.52407244e-08,
        1.62181633e-08, 1.65352813e-08, 1.69391829e-08, 1.77711529e-08,
        1.74490450e-08, 1.81125201e-08, 1.76079597e-08, 1.77025560e-08,
        1.78203642e-08, 1.79140791e-08, 1.80350592e-08, 1.82216479e-08,
        1.83696032e-08, 1.95636384e-08, 1.98517693e-08, 2.10776910e-08,
        2.08502874e-08, 2.08580420e-08, 2.09619874e-08, 2.10964739e-08,
        2.13157372e-08, 2.19395094e-08, 2.26395669e-08, 2.34235183e-08,
        2.42288527e-08, 2.47475332e-08, 2.55255356e-08, 2.621918

In [74]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1)

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [ ]:
from scipy.optimize import curve_fit

# Fonction quadratique pour l'ajustement
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
popt, pcov = curve_fit(quadratic, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Création de la fonction quadratique de l'ajustement
quadratic_fit = np.poly1d(coeffs)

# Génération des valeurs ajustées pour afficher la courbe
fitted_msd = quadratic_fit(time_intervals)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {coeff_uncertainties[1]:.2e}")




Coefficients du polynôme ajusté :
[ 2.03533032e-11  4.01431273e-10 -4.09337786e-12]
Matrice de covariance des coefficients :
[[ 4.54831244e-25 -2.54230941e-24  2.68116159e-24]
 [-2.54230941e-24  1.48607192e-23 -1.65555058e-23]
 [ 2.68116159e-24 -1.65555058e-23  2.07414556e-23]]
Incertitudes sur les coefficients : [6.74411776e-13 3.85496033e-12 4.55427882e-12]
Taille de la particule (m): 2.19e-09 m, avec une incertitude : 3.85e-12


In [91]:
print(f"Quadratic fit coefficients: {coeffs}")

r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Diffusion coefficient
print(f"Taille de la particule (m): {r} m, avec une incertitude : {coeff_uncertainties[0]}")


Quadratic fit coefficients: [ 2.03979100e-11  4.01189164e-10 -3.85706248e-12]
Taille de la particule (m): 2.189828054477303e-09 m, avec une incertitude : [0.00222024]


## anim crop

In [77]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()